In [2]:
import openai
import os
import pandas as pd
import sqlalchemy

In [3]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
df = pd.read_csv("sales_data_sample.csv")

In [4]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [26]:
# df.groupby("QTR_ID").sum()['SALES']

# What was the total amount of sales per quarter?
# select sum(sales) from WHER...

In [5]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [28]:
# Create a TEMP DB in Ram
# Push pandas DF --> DB
# SQL query on this DB

In [6]:
temp_db = create_engine('sqlite:///:memory:',echo=True) # creating a temporary database in our memory

In [7]:
data = df.to_sql(name='Sales',con=temp_db) # converting our pandas df to sql with connection to temp_db with name sales

2023-05-25 12:14:55,324 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-05-25 12:14:55,325 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 12:14:55,326 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-05-25 12:14:55,326 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 12:14:55,329 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 12:14:55,330 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20

In [10]:
with temp_db.connect() as conn:
    # this makes the connection
    # runt code 
    result = conn.execute(text("SELECT SUM(SALES) from Sales")) # pass some sort of sql query to fetch the data
    # auto closes the connection

2023-05-25 12:17:05,365 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) from Sales
2023-05-25 12:17:05,367 INFO sqlalchemy.engine.Engine [generated in 0.00171s] ()


In [11]:
result.all()

[(10032628.85000001,)]

In [12]:
### sqlite sql table with its properties
#
#   Sales({})
#
###

In [16]:
# create a fucntion to create the prompt
def create_table_definition(df):
    # creating a codeblock
    prompt = """ ### sqlite sql table with properties:
    #
    # Sales({})
    #
    """.format(",".join(str(col) for col in df.columns))

    return prompt

In [17]:
print(create_table_definition(df))

 ### sqlite sql table with properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    #
    


In [18]:
def prompt_input():
    nlp_text = input("Enter your search request: ")
    return nlp_text

In [19]:
prompt_input()

'grab all sales in every quarter '

In [20]:
def combine_prompts(df,query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A query to answer: {query_prompt}\n SELECT"
    return definition+query_init_string

In [21]:
nlp_text = prompt_input() # NLP
combine_prompts(df,nlp_text) # Definition of the table + query + NLP 

' ### sqlite sql table with properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    #\n    ### A query to answer: grab all sales from every quarter\n SELECT'

In [22]:
print(combine_prompts(df,nlp_text))

 ### sqlite sql table with properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    #
    ### A query to answer: grab all sales from every quarter
 SELECT


In [32]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=combine_prompts(df,nlp_text),
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=['#',';']
)

In [37]:
response['choices'][0]['text']

' * FROM Sales WHERE QTR_ID IS NOT NULL'

In [38]:
def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(" "):
        query = "SELECT" + query
    return query

In [39]:
with temp_db.connect() as conn:
    # this makes the connection
    # runt code 
    result = conn.execute(text(handle_response(response))) # pass some sort of sql query to fetch the data
    # auto closes the connection

2023-05-25 13:55:37,555 INFO sqlalchemy.engine.Engine SELECT * FROM Sales WHERE QTR_ID IS NOT NULL
2023-05-25 13:55:37,556 INFO sqlalchemy.engine.Engine [generated in 0.00202s] ()


In [41]:
result.all()

[(0, 10107, 30, 95.7, 2, 2871.0, '2/24/2003 0:00', 'Shipped', 1, 2, 2003, 'Motorcycles', 95, 'S10_1678', 'Land of Toys Inc.', '2125557818', '897 Long Airport Avenue', None, 'NYC', 'NY', '10022', 'USA', None, 'Yu', 'Kwai', 'Small'),
 (1, 10121, 34, 81.35, 5, 2765.9, '5/7/2003 0:00', 'Shipped', 2, 5, 2003, 'Motorcycles', 95, 'S10_1678', 'Reims Collectables', '26.47.1555', "59 rue de l'Abbaye", None, 'Reims', None, '51100', 'France', 'EMEA', 'Henriot', 'Paul', 'Small'),
 (2, 10134, 41, 94.74, 2, 3884.34, '7/1/2003 0:00', 'Shipped', 3, 7, 2003, 'Motorcycles', 95, 'S10_1678', 'Lyon Souveniers', '+33 1 46 62 7555', '27 rue du Colonel Pierre Avia', None, 'Paris', None, '75508', 'France', 'EMEA', 'Da Cunha', 'Daniel', 'Medium'),
 (3, 10145, 45, 83.26, 6, 3746.7, '8/25/2003 0:00', 'Shipped', 3, 8, 2003, 'Motorcycles', 95, 'S10_1678', 'Toys4GrownUps.com', '6265557265', '78934 Hillside Dr.', None, 'Pasadena', 'CA', '90003', 'USA', None, 'Young', 'Julie', 'Medium'),
 (4, 10159, 49, 100.0, 14, 5205